In [1]:
!pip install scipy numpy pandas pyarrow
!pip install trl transformers accelerate git+https://github.com/huggingface/peft.git -Uqqq
!pip install datasets bitsandbytes einops wandb -Uqqq

In [2]:
from datasets import load_dataset

dataset_name = "CarperAI/openai_summarize_tldr"
dataset = load_dataset(dataset_name, split="train")

In [3]:
# created with help of https://huggingface.co/docs/datasets/v2.14.1/process#map
def join_prompt_label(example):
    example["text"] = example["prompt"] + " SUMMARY: " + example["label"]
    return example

In [4]:
finetune_dataset = dataset.map(join_prompt_label)

In [5]:
# check a row
finetune_dataset[1]["text"]

'SUBREDDIT: r/loseit\nTITLE: SV & NSV! Keeping on keeping on.\nPOST: 30F, 5\'6". SW: 236 GW: 150 CW: 219\n\nI weigh myself weekly and measure myself monthly. I\'d hit a plateau the last four weeks or so where I was stuck at 222. Felt like kind of a bummer, but knew it\'s because I haven\'t been as strict as I should with my diet, and the last week and a half have been crazy with life things, so I haven\'t been exercising as frequently as I\'ve gotten used to. When I weighed myself as normal on Monday, I was kind of disappointed to see the scale not budging and figured it was time to buckle down again and really watch my diet. Today was my measure-in day, and I\'ve felt cruddy in general since Monday because I caught some chest congestion/cold bug over the weekend. I get on the scale...it says 219. Whaaaaat? I take my measurements, which are down slightly from last month, and with an total-body loss of 8 inches from my starting point on 12/23/14! Some of my clothes have been feeling a b

In [6]:
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [7]:
%pip install accelerate

Note: you may need to restart the kernel to use updated packages.


In [15]:
%pip install git+https://github.com/huggingface/peft.git

  Cloning https://github.com/huggingface/peft.git to /private/var/folders/v5/07ds_8tn6015w7ym01rdgzq00000gq/T/pip-req-build-sht3f8j5
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /private/var/folders/v5/07ds_8tn6015w7ym01rdgzq00000gq/T/pip-req-build-sht3f8j5
  Resolved https://github.com/huggingface/peft.git to commit 9b5808938f28b4357057ac46a7359b70cd2997c8
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for peft: filename=peft-0.5.0.dev0-py3-none-any.whl size=73785 sha256=6947b657e193d2089e857a1e55b8c94d650720ca7b8a09de1e0d24f4bccfbfee
  Stored in directory: /private/var/folders/v5/07ds_8tn6015w7ym01rdgzq00000gq/T/pip-ephem-wheel-cache-2qunuasn/wheels/d7/c7/de/1368fac8590e1b103ddc2ec2a28ad51d83aded1a3830e8a087
Successfully built peft
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install datasets bitsandbytes einops wandb -Uqqq

Note: you may need to restart the kernel to use updated packages.


In [11]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

model_name = "ybelkada/falcon-7b-sharded-bf16"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map={"": "mps"},  # https://huggingface.co/spaces/cai/llama-lora-finetune-cn/blob/main/app.py#L23
)
model.config.use_cache = False

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [12]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

In [16]:
from peft import LoraConfig, get_peft_model, PeftConfig, PeftModel, prepare_model_for_kbit_training

In [17]:
model = prepare_model_for_kbit_training(model)

lora_alpha = 32
lora_dropout = 0.05
lora_r = 32

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[         # Setting the Falcon modules that we want to fine-tune as `target_modules`
        "query_key_value",
        "dense",
        "dense_h_to_4h",
        "dense_4h_to_h",
    ]
)

peft_model = get_peft_model(model, peft_config)

In [18]:
from transformers import TrainingArguments

In [19]:
!mkdir ./falcon-7b-finetuned-openai_summarize_tldr

In [21]:
output_dir = "./falcon-7b-finetuned-openai_summarize_tldr"
per_device_train_batch_size = 4 # reduce batch size by 2x if out-of-memory error
gradient_accumulation_steps = 8  # increase gradient accumulation steps by 2x if batch size is reduced
optim = "paged_adamw_32bit" # activates the paging for better memory management
num_train_epochs = 1
save_steps = 10
logging_steps = 10  # log training loss after every 10 steps
learning_rate = 2e-4  # learning rate
max_grad_norm = 0.3
max_steps = 180        # training will happen for 180 steps
warmup_ratio = 0.03
lr_scheduler_type = "cosine"  # learning rate scheduler

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    num_train_epochs=num_train_epochs,
    fp16=False,        # FP16 Mixed precision training can only be used on CUDA devices.
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type,
    push_to_hub=True,
)

In [23]:
%pip install trl

  Obtaining dependency information for trl from https://files.pythonhosted.org/packages/cd/8a/190dbc1a945fde9d1136073d6492290aa29aab209cfdab2faf427d1ed88b/trl-0.5.0-py3-none-any.whl.metadata
  Using cached trl-0.5.0-py3-none-any.whl.metadata (10 kB)
Using cached trl-0.5.0-py3-none-any.whl (88 kB)
Note: you may need to restart the kernel to use updated packages.


In [24]:
from trl import SFTTrainer

In [25]:
# need to subscribe to huggingface and generate a read token at https://huggingface.co/settings/tokens
from huggingface_hub import notebook_login
notebook_login()

In [26]:
# need to manually create ftrojan/falcon-7b-finetuned-openai_summarize_tldr in the huggingface web
trainer = SFTTrainer(
    model=peft_model,
    train_dataset=finetune_dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=512,
    tokenizer=tokenizer,
    args=training_arguments,
)

Map:   0%|          | 0/116722 [00:00<?, ? examples/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Cloning https://huggingface.co/ftrojan/falcon-7b-finetuned-openai_summarize_tldr into local empty directory.


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [27]:
!export TOKENIZERS_PARALLELISM=true

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [30]:
# upcasting the layer norms in torch.bfloat16 for more stable training
# TypeError: Trying to convert BFloat16 to the MPS backend but it does not have support for that dtype.
# for name, module in trainer.model.named_modules():
#     if "norm" in name:
#         module = module.to(torch.bfloat16)

In [31]:
# authenticate WandB for logging metrics
import wandb
wandb.login()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/ftrojan/.netrc


True

16:50

In [32]:
%%time
peft_model.config.use_cache = False
trainer.train()

wandb: Currently logged in as: ftrojan (oracle-jarvis). Use `wandb login --relogin` to force relogin


You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


FP4 quantization state not initialized. Please call .cuda() or .to(device) on the LinearFP4 layer first.


AssertionError: 

In [ ]:
trainer.push_to_hub()